In [1]:
import os
import torch
import json
import yaml
import tqdm
from dataset.ConcatBaselineDataset import ConcatBaselineDataset
from baseline.models.ConcatBaselineNet import ConcatBaselineNet
from dataset.MurelNetDataset import MurelNetDataset
from murel.models.MurelNet import MurelNet
from baseline.scripts.train_baseline import get_hidden_layer_list
from torch.utils.data import DataLoader
import transforms.transforms as trfm

In [4]:
DATA_DIR = "/auto/homes/bat34/VQA/"
ROOT_DIR = "/auto/homes/bat34/VQA_PartII/"
with open(os.path.join(ROOT_DIR, 'murel', 'scripts', 'murel.yaml')) as f:
        config = yaml.load(f)
config = config['murel_options']
test_dataset = MurelNetDataset(split='test', \
                        txt_enc=config['txt_enc'], \
                        bottom_up_features_dir=config['bottom_up_features_dir'], \
                        skipthoughts_dir=config['skipthoughts_dir'], \
                        processed_dir=config['processed_dir'], \
                        ROOT_DIR=ROOT_DIR, \
                        vqa_dir=config['vqa_dir'])


test_loader = DataLoader(test_dataset, shuffle=False, \
                         num_workers=config['num_workers'], \
                         batch_size=config['batch_size']
)
model = MurelNet(config)

#Path to model weights with best performance on the validation dataset
model.load_state_dict(torch.load(os.path.join(ROOT_DIR, 'murel', 'trained_models', \
                                   'murel_txt_enc_BayesianUniSkip_batch_size_84_lr_0.0003_lr_decay_rate_0.25_unroll_steps_3_fusion_type_block_accuracy_0.5468845330673355_epoch_23_BEST.pth')))
model.cuda()
model.eval()

/home/bat34/venv/lib/python3.5/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


MurelNet(
  (murel_cell): MurelCell(
    (fusion_features): Block(
      (linear0): Linear(in_features=2048, out_features=1000, bias=True)
      (linear1): Linear(in_features=2400, out_features=1000, bias=True)
      (merge_linears0): ModuleList(
        (0): Linear(in_features=50, out_features=750, bias=True)
        (1): Linear(in_features=50, out_features=750, bias=True)
        (2): Linear(in_features=50, out_features=750, bias=True)
        (3): Linear(in_features=50, out_features=750, bias=True)
        (4): Linear(in_features=50, out_features=750, bias=True)
        (5): Linear(in_features=50, out_features=750, bias=True)
        (6): Linear(in_features=50, out_features=750, bias=True)
        (7): Linear(in_features=50, out_features=750, bias=True)
        (8): Linear(in_features=50, out_features=750, bias=True)
        (9): Linear(in_features=50, out_features=750, bias=True)
        (10): Linear(in_features=50, out_features=750, bias=True)
        (11): Linear(in_features=50, 

In [ ]:
results = []
with torch.no_grad():
    for data in tqdm.tqdm(test_loader):
        item = {\
        'question_embedding': data['question_embedding'].cuda(), \
        'object_features_list': data['object_features_list'].cuda(), \
        'bounding_boxes': data['bounding_boxes'].cuda(), \
        }
        inputs = item
        qids = data['question_id']
        outputs = model(inputs)
        values, ans_indices = torch.max(outputs, dim=1)
        ans_indices = list(ans_indices)
        ans_indices = [tsr.item() for tsr in ans_indices]
        for qid, ans_idx in zip(qids, ans_indices):
            results.append({
                'question_id': int(qid),
                'answer': test_dataset.aid_to_ans[ans_idx]
            })

  0%|          | 0/5331 [00:00<?, ?it/s]/home/bat34/venv/lib/python3.5/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/bat34/venv/lib/python3.5/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/bat34/venv/lib/python3.5/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/bat34/venv/lib/python3.5/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/bat34/venv/lib/pyt

In [ ]:
with open(os.path.join(ROOT_DIR, 'murel', 'murel_test-2015_results.json'), 'w') as f:
    json.dump(results, f)